In [30]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [31]:
!git clone https://github.com/jwkirchenbauer/lm-watermarking

fatal: destination path 'lm-watermarking' already exists and is not an empty directory.


In [32]:
import os
os.chdir("/kaggle/working/lm-watermarking")

In [33]:
!pip install torch transformers accelerate optimum

In [34]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
from tqdm import tqdm
from optimum.bettertransformer import BetterTransformer

In [35]:
from extended_watermark_processor import WatermarkLogitsProcessor
from transformers import LogitsProcessor,LogitsProcessorList

# model_name = "facebook/opt-350m"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
# watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
#                                                gamma=0.25,
#                                                delta=2.0,
#                                                seeding_scheme="selfhash") #equivalent to `ff-anchored_minhash_prf-4-True-15485863`
# # Note:
# # You can turn off self-hashing by setting the seeding scheme to `minhash`.
# input_text = "A good thesis"
# tokenized_input = tokenizer(input_text, return_tensors='pt').to(model.device)
# # note that if the model is on cuda, then the input is on cuda
# # and thus the watermarking rng is cuda-based.
# # This is a different generator than the cpu-based rng in pytorch!

# output_tokens = model.generate(**tokenized_input,
#                                logits_processor=LogitsProcessorList([watermark_processor]))

# # if decoder only model, then we need to isolate the
# # newly generated tokens as only those are watermarked, the input/prompt is not
# output_tokens = output_tokens[:,tokenized_input["input_ids"].shape[-1]:]

# output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)[0]

In [36]:
# print(output_text)

In [37]:

from extended_watermark_processor import WatermarkDetector

# watermark_detector = WatermarkDetector(vocab=list(tokenizer.get_vocab().values()),
#                                         gamma=0.25, # should match original setting
#                                         seeding_scheme="selfhash", # should match original setting
#                                         device=model.device, # must match the original rng device type
#                                         tokenizer=tokenizer,
#                                         z_threshold=4.0,
#                                         normalizers=[],
#                                         ignore_repeated_ngrams=True)

# score_dict = watermark_detector.detect(output_text) # or any other text of interest to analyze

In [ ]:
# print(score_dict)

In [38]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed,AutoModelForSeq2SeqLM
from datasets import load_dataset
# from tqdm import tqdm
from optimum.bettertransformer import BetterTransformer
from tqdm.auto import tqdm  
import os, argparse

In [40]:
!pip install undecorated

In [43]:
from undecorated import undecorated
from types import MethodType


In [ ]:
# # Load your model and tokenizer
# # model_name = "facebook/opt-350m"
# # tokenizer = AutoTokenizer.from_pretrained(model_name)
# # model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")

# model_name = "facebook/opt-350m"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).to("cuda")
# watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
#                                                gamma=0.25,
#                                                delta=2.0,
#                                                seeding_scheme="selfhash")

# correct = 0  # Number of correct predictions
# total = 0  # Total number of predictions

# # Open the HellaSwag jsonl file
# with open('/kaggle/working/hellaswag/data/hellaswag_val.jsonl', 'r') as f:
#   # Loop through each line in the file
#   for line in tqdm(f, desc="Evaluating"):
#     item = json.loads(line)  # Parse each line as a JSON object

#     context = item['ctx']  # Use 'ctx' field as context
#     endings = item['endings']  # Possible endings
#     correct_end = item['label']  # Correct answer index is in 'label' field

#     # Generate a score for each possible ending
#     scores = []
#     for ending in endings:
#         input_text = context + ' ' + ending  # Combine context and ending
#         input_ids = tokenizer.encode(input_text, return_tensors='pt').to("cuda")  # Tokenize input
        
#         with torch.no_grad():
#             outputs = model(input_ids, labels=input_ids)  # Get model output
#             loss = outputs.loss  # Get loss value
#             scores.append(loss.item())  # Add loss value to scores list

#     # Choose the most likely ending based on the scores (lowest loss)
#     predicted_end = scores.index(min(scores))

#     # Update the number of correct predictions
#     correct += 1 if predicted_end == correct_end else 0
#     total += 1  # Update the total number of predictions

# # Print the final accuracy after the loop
# print(f"Final Accuracy: {correct / total:.4f}")


In [44]:
os.chdir("/kaggle/working/")

In [45]:
!git clone https://github.com/rowanz/hellaswag

Cloning into 'hellaswag'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 45 (delta 0), reused 2 (delta 0), pack-reused 41
Unpacking objects: 100% (45/45), 17.69 MiB | 8.21 MiB/s, done.


In [64]:
# Load your model and tokenizer
model_name = "facebook/opt-350m"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")
# Define watermark processor
watermark_processor = WatermarkLogitsProcessor(vocab=list(tokenizer.get_vocab().values()),
                                               gamma=0.25,
                                               delta=2.0,
                                               seeding_scheme="selfhash")

correct = 0  # Number of correct predictions
total = 0  # Total number of predictions

# Open the HellaSwag jsonl file
with open('/kaggle/working/hellaswag/data/hellaswag_val.jsonl', 'r') as f:
  # Loop through each line in the file
  for line in tqdm(f, desc="Evaluating"):
    item = json.loads(line)  # Parse each line as a JSON object

    context = item['ctx']  # Use 'ctx' field as context
    endings = item['endings']  # Possible endings
    correct_end = item['label']  # Correct answer index is in 'label' field

    # Generate scores for each possible ending with watermarking
    scores = []
    for ending in endings:
        input_text = context + ' ' + ending  # Combine context and ending

        # Tokenize input
        tokenized_input = tokenizer(input_text, return_tensors='pt').to(model.device)

        # Generate with watermark embedding
        with torch.no_grad():
            output_tokens = model.generate(**tokenized_input,
                                        logits_processor=LogitsProcessorList([watermark_processor]),max_length =100, return_dict_in_generate=True, output_scores=True)
            print(type(output_tokens))
        # Isolate newly generated tokens (decoder only model)
            if isinstance(model, AutoModelForSeq2SeqLM):
                output_tokens = output_tokens[:, tokenized_input["input_ids"].shape[-1]:]

        # Get loss (assuming lower loss = better fit)
#         print(output_tokens.scores)
        loss = output_tokens.sequences
        scores.append(loss)

    # Choose most likely ending based on the scores (lowest loss)
    predicted_end = scores.index(min(scores))

    # Update accuracy metrics
    correct += 1 if predicted_end == correct_end else 0
    total += 1
    print("Accuracy({}/{}): {}".format(correct, total, correct / total), end='\r')
# Print the final accuracy after the loop
print(f"Final Accuracy: {correct / total:.4f}")


Evaluating: 0it [00:00, ?it/s]

<class 'transformers.generation.utils.GenerateDecoderOnlyOutput'>
<class 'transformers.generation.utils.GenerateDecoderOnlyOutput'>
<class 'transformers.generation.utils.GenerateDecoderOnlyOutput'>
<class 'transformers.generation.utils.GenerateDecoderOnlyOutput'>


RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [60]:
# print(output_tokens)